# Introduction
Text generation is another ability of RNN. In this part, we will find out how to create a generative model for text. In detail, a LSTM network which we will build learn sequences of characters from Alice in Wonderland.


Dataset: we choose a dataset from familiar novel [Alice’s Adventures in Wonderland](http://www.gutenberg.org/cache/epub/11/pg11.txt) by Lewis Carroll



## Importing the classes and functions

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys

Using TensorFlow backend.


## Load dataset

In [2]:
filename = "pg11.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
#convert all of the characters to lowercase to reduce the vocabulary that the network must learn

## Convert into integer


In [3]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

print("chars")
print(chars)
print("char_to_int")
print(char_to_int)

chars
['\n', '\r', ' ', '!', '#', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\x80', '\x98', '\x99', '\x9c', '\x9d', '\xbb', '\xbf', '\xe2', '\xef']
char_to_int
{'\x80': 57, '\n': 0, '\r': 1, '\xef': 65, '\x99': 59, '\x98': 58, '\x9d': 61, '\x9c': 60, '!': 3, ' ': 2, '#': 4, '%': 6, '$': 5, '\xbf': 63, ')': 8, '(': 7, '*': 9, '-': 11, ',': 10, '/': 13, '.': 12, '1': 15, '0': 14, '3': 17, '2': 16, '5': 19, '4': 18, '7': 21, '6': 20, '9': 23, '8': 22, ';': 25, ':': 24, '?': 26, '@': 27, '\xe2': 64, '[': 28, ']': 29, '\xbb': 62, '_': 30, 'a': 31, 'c': 33, 'b': 32, 'e': 35, 'd': 34, 'g': 37, 'f': 36, 'i': 39, 'h': 38, 'k': 41, 'j': 40, 'm': 43, 'l': 42, 'o': 45, 'n': 44, 'q': 47, 'p': 46, 's': 49, 'r': 48, 'u': 51, 't': 50, 'w': 53, 'v': 52, 'y': 55, 'x': 54, 'z': 56}


## Check size of dataset


In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

Total Characters:  173595
Total Vocab:  66


## Create data for training 

We will split the book text up into subsequences with a fixed length(here we choose 100 character). Each training pattern of the network is comprised of 100 time steps of one character (X) followed by one character output (y). When creating these sequences, we slide this window along the whole book one character at a time. 
Let see a simple example with sequence's length is 5
- CHAPT -> E
- HAPTE -> R

In [5]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  173495


## Prepare data for training 
- Tranform the list of input into the form expected by an LTSM network. 
- As usual, we rescale the number into range 0 to 1 to easy training network.
- Convert the output parterns into one hot encoding which we can configure the network to predict the probalities of 62 characters. Each y value is converted into a sparse vector with a length of 62, full of zeros except with a 1 in the column for the letter (integer) that the pattern represents. For example: 
(
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
)


In [6]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

## Create model 

In [ ]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

## Training created model 

In [ ]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs=10, batch_size=128, callbacks=callbacks_list)

Epoch 1/10
173495/173495 [==============================] - 2595s - loss: 3.0241  
Epoch 2/10
173495/173495 [==============================] - 2785s - loss: 2.6937  
Epoch 3/10
 21376/173495 [==>...........................] - ETA: 2289s - loss: 2.6115

## Test

In [ ]:
# create input data
int_to_char = dict((i, c) for i, c in enumerate(chars))
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""

In [ ]:
#Generate text using trained model
for i in range(50):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."